In [2]:
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
from torch.optim import AdamW

In [6]:
from torch.utils.data import Dataset, DataLoader

In [7]:
encoder = LabelEncoder()

In [8]:
df = pd.read_csv('../../dataset/with_prefix.csv')

In [9]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,name,description,response,natasha_response,class,natasha_response_stopwords
0,0,0,0,13246,Финансовый аналитик,обязанности организовывает управление движение...,организовывает управление движением финансовых...,организовывать управление движение финансовый ...,0,организовывать управление движение финансовый ...
1,1,1,1,14266,Финансовый менеджер,обязанности составление бюджета доходов бюджет...,"Составление бюджета доходов и расходов, контро...","составление бюджет доход и расход , контроль и...",0,составление бюджет доход расход исполнение при...
2,2,2,2,14674,Финансовый аналитик,уважаемый кандидат компания alem sauda ltd рад...,Разработка и внедрение внутренних регламентиру...,разработка и внедрение внутренний регламентиро...,0,разработка внедрение внутренний регламентирова...
3,3,3,3,15042,Старший финансовый аналитик,требования высшее финансовое экономическое обр...,Осуществлять работу экономическому планировани...,осуществлять работа экономический планирование...,0,осуществлять экономический ирование выявлять и...
4,4,4,4,16179,Финансовый аналитик на проект Sandyq,обязанности формирование управленческой отчетн...,"формирование управленческой отчетности, анализ...","формирование управленческий отчетность , анали...",0,формирование управленческий отчетность анализ ...
...,...,...,...,...,...,...,...,...,...,...
16897,16897,16897,17636,82155,Снабженец-логист,обязанности организация закупки соответствующи...,"Организация закупки товаров, организация долго...","организация закупка товар , организация долгос...",75,закупка товар долгосрочный сотрудничество пост...
16898,16898,16898,17637,82695,Логист,обязанности поиск автотранспорта перевозкам рф...,"Поиск автотранспорта, перевозки РФ РК внутри Р...","поиск автотранспорт , перевозка рф рк внутри р...",75,поиск автотранспорт перевозка рф рк внутри рк ...
16899,16899,16899,17638,82941,Специалист по логистике,обязанности отвечать звонки обрабатывать заявк...,"Отвечать звонки, обрабатывать заявки, презента...","отвечать звонок , обрабатывать заявка , презен...",75,отвечать звонок обрабатывать заявка презентаци...
16900,16900,16900,17639,83572,Специалист по логистике,обязанности организация планирование контроль ...,"Организация, планирование, контроль и регулиро...","организация , планирование , контроль и регули...",75,ирование регулирование доставка товар оптималь...


In [8]:
filtered_row = df[df['class'] == 0].head(1)

In [9]:
filtered_row['natasha_response_stopwords'][0]

'организовывать управление движение финансовый ресурс регулирование финансовый отношение разработка норматив оборотный средство движение финансовый средство составление отчетность правильность составление своевременность предоставление отчетный рациональный денежный средство минимизация расход финансовый экономический стаж область финансовый деятельность обязательно Финансовый аналитик'

In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = AutoModelForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=76, output_attentions=True, output_hidden_states=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/opt/jupyterhub/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [11]:
def tokenize_function(description, tokenizer, max_length=512):
    return tokenizer(description.tolist(), padding='max_length', truncation=True, max_length=max_length)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df['natasha_response_stopwords'], df['class'], test_size=0.2, random_state=42)

In [13]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [14]:
X_train_list = X_train.astype(str).tolist()
X_test_list = X_test.astype(str).tolist()

# Define your tokenize function (without the .tolist() call)
def tokenize_function(description, tokenizer, max_length=512):
    return tokenizer(description, padding='max_length', truncation=True, max_length=max_length)

# Tokenize the lists of strings
train_encodings = tokenize_function(X_train_list, tokenizer)
test_encodings = tokenize_function(X_test_list, tokenizer)

In [15]:
train_labels = torch.tensor(y_train.values)
test_labels = torch.tensor(y_test.values)

In [16]:
class JobDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [17]:
train_dataset = JobDataset(train_encodings, y_train)
test_dataset = JobDataset(test_encodings, y_test)

In [18]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [19]:
optimizer = AdamW(model.parameters(), lr=2e-5)

In [20]:
def save_checkpoint(model, optimizer, epoch, path="checkpoint.pth"):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch
    }
    torch.save(checkpoint, path)
    print(f"Сохранен snapshot на эпохе {epoch} в {path}")

In [21]:
def load_checkpoint(path, model, optimizer):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    print(f"Загружен snapshot с {epoch}-й эпохи")
    return epoch

In [22]:
from tqdm import tqdm

def train_model(model, train_loader, test_loader, optimizer, device, num_epochs=7, gradient_accumulation_steps=4, save_path="checkpoint.pth"):
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        optimizer.zero_grad()  

        train_loader_tqdm = tqdm(train_loader, desc=f"Эпоха {epoch+1}/{num_epochs} - Тренировка", leave=False)

        for step, batch in enumerate(train_loader_tqdm):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss = loss / gradient_accumulation_steps  
            total_train_loss += loss.item()

            loss.backward()

            if (step + 1) % gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()  

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        model.eval()
        total_val_loss = 0

        test_loader_tqdm = tqdm(test_loader, desc=f"Эпоха {epoch+1}/{num_epochs} - Валидация", leave=False)
        with torch.no_grad():
            for batch in test_loader_tqdm:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(test_loader)
        val_losses.append(avg_val_loss)

        print(f"Эпоха {epoch+1}/{num_epochs} | Тренировочная потеря: {avg_train_loss:.4f} | Валид потеря: {avg_val_loss:.4f}")
        
        save_checkpoint(model, optimizer, epoch+1, path=f"{save_path}_epoch_{epoch+1}.pth")

    return train_losses, val_losses

In [23]:
save_directory = "../../snapshoot/model.pth"

In [24]:
train_losses, val_losses = train_model(model, train_loader, test_loader, optimizer, device, num_epochs=5, gradient_accumulation_steps=4, save_path="checkpoint")

Эпоха 1/5 - Тренировка:   0%|          | 0/1691 [00:00<?, ?it/s]BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Эпоха 1/5 | Тренировочная потеря: 0.6686 | Валид потеря: 1.3193
Сохранен snapshot на эпохе 1 в checkpoint_epoch_1.pth


Эпоха 2/5 | Тренировочная потеря: 0.2447 | Валид потеря: 0.7359
Сохранен snapshot на эпохе 2 в checkpoint_epoch_2.pth


Эпоха 3/5 | Тренировочная потеря: 0.1458 | Валид потеря: 0.5883
Сохранен snapshot на эпохе 3 в checkpoint_epoch_3.pth


Эпоха 4/5 | Тренировочная потеря: 0.1060 | Валид потеря: 0.5850
Сохранен snapshot на эпохе 4 в checkpoint_epoch_4.pth


Эпоха 5/5 | Тренировочная потеря: 0.0804 | Валид потеря: 0.5547
Сохранен snapshot на эпохе 5 в checkpoint_epoch_5.pth


In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model, test_loader, device):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=-1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='weighted')  
    recall = recall_score(all_labels, all_predictions, average='weighted')       
    f1 = f1_score(all_labels, all_predictions, average='weighted')                

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1-score: {f1:.4f}')

evaluate_model(model, test_loader, device)


Accuracy: 0.8524
Precision: 0.8577
Recall: 0.8524
F1-score: 0.8525


In [26]:
epoch = load_checkpoint('checkpoint_epoch_3.pth', model, optimizer)

/tmp/ipykernel_23699/2814753524.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


Загружен snapshot с 3-й эпохи


In [27]:
evaluate_model(model, test_loader, device)

Accuracy: 0.8388
Precision: 0.8402
Recall: 0.8388
F1-score: 0.8335


In [28]:
test_data = pd.read_csv('../../dataset/test_data_0_406.csv')

In [29]:
from tqdm import tqdm
import torch
import torch.nn.functional as F
import pandas as pd


def tokenize_descriptions(descriptions, tokenizer, max_length=512):
    return tokenizer(descriptions.tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")

encodings = tokenize_descriptions(test_data['clean_description'], tokenizer)

class DescriptionDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

description_dataset = DescriptionDataset(encodings)
description_loader = DataLoader(description_dataset, batch_size=16, shuffle=False)

def predict(model, data_loader, device):
    model.eval()
    predictions = []
    probabilities = []
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Предсказания"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            
            probs = F.softmax(logits, dim=-1)
            
            predicted_classes = torch.argmax(probs, dim=-1)
            
            predictions.extend(predicted_classes.cpu().numpy())
            probabilities.extend(probs.cpu().numpy())
    
    return predictions, probabilities
epoch = load_checkpoint('checkpoint_epoch_3.pth', model, optimizer)
predicted_classes, predicted_probabilities = predict(model, description_loader, device)

test_data['predicted_class'] = predicted_classes
test_data['predicted_probability'] = [max(prob) for prob in predicted_probabilities]


/tmp/ipykernel_23699/2814753524.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


Загружен snapshot с 3-й эпохи


Предсказания: 100%|██████████| 4343/4343 [32:02<00:00,  2.26it/s]


In [30]:
sample = test_data.groupby(['predicted_class']).mean('predicted_probability')

In [31]:
sample

,Unnamed: 0,id,class,predicted_probability
predicted_class,,,,
0,26751.000000,7.805280e+05,108.000000,0.188523
1,23828.923077,5.237138e+05,85.461538,0.560894
2,23949.060665,8.976478e+05,109.365949,0.693482
3,20416.294372,6.802178e+05,80.969697,0.374523
4,29493.571429,1.046879e+06,141.285714,0.397105
...,...,...,...,...
71,60756.373164,7.969419e+05,78.110676,0.606423
72,59690.582463,6.520419e+05,72.757829,0.279594
73,61027.628431,5.846800e+05,74.787382,0.402610


In [35]:
sample.to_excel('asa.xlsx')

In [38]:
test_data['predicted_class'].value_counts().sort_index().index

Index([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
       55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
       73, 74, 75],
      dtype='int64', name='predicted_class')